In [1]:
from solver import Span, Beam

In [2]:
J = (0.3 * 0.5**3)/12 # m4
EJ =  31476*1000000*J/1000 # Mpa * m4 -> N*m2 -> kN*m2

c_1 = Span(lenght = 3.00, ej = EJ)
c_2 = Span(lenght = 4.50, ej = EJ)
c_3 = Span(lenght = 4.00, ej = EJ)
c_4 = Span(lenght = 5.00, ej = EJ)
c_5 = Span(lenght = 6.15, ej = EJ)
c_6 = Span(lenght = 4.00, ej = EJ)

trave = Beam(spans = [c_1, c_2, c_3, c_4, c_5, c_6])

In [3]:
print(f"{trave.spans_lenght() = }")
print(f"{trave.spans_total_lenght() = }")
print(f"{trave.spans_cum_lenght() = }")
print(f"{trave.spans_ej() = }")
print(f"{trave.spans_q_max() = }")
print(f"{trave.spans_q_min() = }")


trave.spans_lenght() = [3.0, 4.5, 4.0, 5.0, 6.15, 4.0]
trave.spans_total_lenght() = 26.65
trave.spans_cum_lenght() = array([ 3.  ,  7.5 , 11.5 , 16.5 , 22.65, 26.65])
trave.spans_ej() = [98362.49999999999, 98362.49999999999, 98362.49999999999, 98362.49999999999, 98362.49999999999, 98362.49999999999]
trave.spans_q_max() = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
trave.spans_q_min() = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [4]:
x1 = Span(1,2)
x2 = Span(5,6)
x3 = Span(1.5, 7.6)
beam = Beam([x1,x2,x3])

for span in beam.spans: 
    print(span.ej)

print(beam.spans_lenght())
print(beam.spans_ej())
print(beam.spans_q_max())
print(beam.spans_q_min())


2
6
7.6
[1, 5, 1.5]
[2, 6, 7.6]
[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]
